# Necessary imports

In [ ]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from main import process_imdb_data
from pyspark.sql import DataFrame
from pyspark.sql.functions import desc

In [ ]:
dataframes = process_imdb_data()

In [ ]:
def export_result(df: DataFrame, result_path: str, show_rows: int = 20, title: str = None,
                   partition_column: str = None, num_partitions: int = None) -> None:
     print(f"Results for: {title}")
     df.show(show_rows, truncate=False)
     total_records = df.count()
     print(f"Total number of records in the result: {total_records}")

     for col_name, col_type in df.dtypes:
         if col_type.startswith("array"):
             df = df.withColumn(col_name, F.concat_ws(", ", F.col(col_name)))

     writer = df.write.option("header", "true")

     if partition_column and partition_column in df.columns:
         print(f"Exporting results with partitioning by column: {partition_column}")
         writer.partitionBy(partition_column).csv(result_path, mode="overwrite")
     elif num_partitions and num_partitions > 0:
         print(f"Exporting results with {num_partitions} partitions")
         df.repartition(num_partitions).write.option("header", "true").csv(result_path, mode="overwrite")
     else:
         target_size_mb = 128
         estimated_size_bytes = total_records * len(df.columns) * 100
         estimated_size_mb = estimated_size_bytes / (1024 * 1024)
         optimal_partitions = max(1, int(estimated_size_mb / target_size_mb))

         print(f"Exporting results with automatically determined {optimal_partitions} partitions")
         df.repartition(optimal_partitions).write.option("header", "true").csv(result_path, mode="overwrite")

# Queries

## №1: Which romantic comedies with Ukrainian localization are the most popular (with a rating of 7+ and at least 1000 votes)?

###  №1: Які романтичні комедії з українською локалізацією найпопулярніші (мають рейтинг 7+ і щонайменше 1000 голосів)?

In [ ]:
def top_ua_romantic_comedies(basics: DataFrame, akas: DataFrame, ratings: DataFrame) -> DataFrame:

    romantic_comedies = basics \
        .filter(
            F.array_contains("genres", "Romance") &
            F.array_contains("genres", "Comedy")
        ) \
        .select("tconst", "primaryTitle", "titleType", "genres")

    ua_titles = akas \
        .filter((F.col("region") == "UA") & (F.col("isOriginalTitle") == False)) \
        .select("titleId", F.col("title").alias("ukrainianTitle")) \
        .distinct()

    rated = ratings \
        .filter(
            (F.col("averageRating") >= 7) &
            (F.col("numVotes") >= 1000)
        )

    result = romantic_comedies \
        .join(ua_titles, romantic_comedies.tconst == ua_titles.titleId, "inner") \
        .join(rated, "tconst") \
        .select(
            "tconst", "primaryTitle", "ukrainianTitle", "titleType",
            "genres", "averageRating", "numVotes"
        ) \
        .orderBy(F.col("averageRating").desc(), F.col("numVotes").desc())

    # result.show(20, truncate=False)
    return result

In [ ]:
result = top_ua_romantic_comedies(dataframes["basics"], dataframes["akas"], dataframes["ratings"])
export_result(result,"/data/results/top_ua_romantic_comedies.csv",title="Top romantic comedies available in Ukrainian")

Results for: Top romantic comedies available in Ukrainian
+----------+----------------------------+-----------------------------------------------------------+------------+----------------------------+-------------+--------+
|tconst    |primaryTitle                |ukrainianTitle                                             |titleType   |genres                      |averageRating|numVotes|
+----------+----------------------------+-----------------------------------------------------------+------------+----------------------------+-------------+--------+
|tt0108778 |Friends                     |Друзі                                                      |tvSeries    |[Comedy, Romance]           |8.9          |1139938 |
|tt5555260 |This Is Us                  |Це ми                                                      |tvSeries    |[Comedy, Drama, Romance]    |8.7          |167375  |
|tt11769304|Hospital Playlist           |Музика лікарні                                             |tvSeri

## №2: Top directors by number of video games directed

### №2: Скільки ігор у доробку кожного режисера?

In [ ]:
def videogames_by_director(df_basics: DataFrame, df_crew: DataFrame, df_name: DataFrame) -> DataFrame:

    video_games_with_crew = df_basics \
        .join(df_crew, "tconst", "inner") \
        .filter(
            (F.col("directors").isNotNull()) &
            (F.col("titleType") == "videoGame")
        )

    video_games_with_directors = video_games_with_crew \
        .withColumn("director", F.explode(F.col("directors")))

    video_games_with_directors_and_names = video_games_with_directors \
        .join(
            df_name,
            video_games_with_directors["director"] == df_name["nconst"],
            "inner"
        )

    director_video_game_count = video_games_with_directors_and_names \
        .groupBy("primaryName") \
        .agg(
            F.count("tconst").alias("num_video_games")
        )

    result = director_video_game_count \
        .orderBy(F.desc("num_video_games"))

    # result.show(20, truncate=False)
    return result

In [ ]:
result = videogames_by_director(dataframes['basics'], dataframes['crew'], dataframes['name'])
export_result(result,"/data/results/videogames_by_director.csv",title="Count of videogames for each game director")

Results for: Count of videogames for each game director
+--------------------------+---------------+
|primaryName               |num_video_games|
+--------------------------+---------------+
|Randall Ryan              |71             |
|Paul Andrews              |68             |
|Lani Minella              |55             |
|Keith Arem                |55             |
|Mo Davoudian              |52             |
|Safak Yalcinkaya          |47             |
|Douglas Carrigan          |46             |
|Satoshi Tajiri            |40             |
|Classtro                  |39             |
|James Deighan             |35             |
|Jon Burton                |35             |
|Hideo Kojima              |34             |
|Blade Zavier              |33             |
|Matt Bozon                |27             |
|Tetsuya Nomura            |25             |
|Todd Howard               |24             |
|Suda 51                   |23             |
|Michael Andrew Lambert Jr.|23             |

## №3: Which video games by Randall Ryan have a rating above 6.5 with over 200 votes?

### №3: Які відеоігри режисера Randall Ryan мають рейтинг вище 6.5 та більше 200 голосів?

Average for rating and votes count | Середні показники для рейтингу та кількості голосів для відеігор.

In [ ]:
videogames = dataframes["basics"].filter(F.col("titleType") == "videoGame") \
    .join(dataframes["ratings"], "tconst", "inner")

videogames.select(
    F.mean("averageRating").alias("avg_rating"),
    F.mean("numVotes").alias("avg_votes")
).show()

+-----------------+------------------+
|       avg_rating|         avg_votes|
+-----------------+------------------+
|6.725510884051531|357.69535761883606|
+-----------------+------------------+



Queiry | Запит:

In [ ]:
def top_titles_by_randall_ryan(df_name: DataFrame, df_crew: DataFrame, df_basics: DataFrame, df_ratings: DataFrame) -> DataFrame:

    randall_nconst_df = df_name.filter(F.col("primaryName") == "Randall Ryan").select("nconst")
    randall_nconst = [row["nconst"] for row in randall_nconst_df.collect()]

    if not randall_nconst:
        print("Randall Ryan not found in df_name.")
        return
    randall_id = randall_nconst[0]

    relevant_titles = df_crew.filter(
        (F.array_contains(F.col("directors"), randall_id)) |
        (F.array_contains(F.col("writers"), randall_id))
    ).select("tconst")

    titles_with_info = relevant_titles \
        .join(df_basics, "tconst", "inner") \
        .join(df_ratings, "tconst", "inner") \
        .filter((F.col("averageRating") >= 6.5) & (F.col("numVotes") >= 200))

    result = titles_with_info.select(
        "primaryTitle", "titleType", "startYear", "averageRating", "numVotes"
    ).orderBy(F.desc("averageRating"), F.desc("numVotes"))

    # result.show(20, truncate=False)
    return result

In [ ]:
result = top_titles_by_randall_ryan(dataframes["name"], dataframes["crew"], dataframes["basics"], dataframes["ratings"])
export_result(result, "/data/results/top_titles_by_randall_ryan.csv", title="Top games by Randall Ryan")

Results for: Top games by Randall Ryan
+----------------------------------------+---------+---------+-------------+--------+
|primaryTitle                            |titleType|startYear|averageRating|numVotes|
+----------------------------------------+---------+---------+-------------+--------+
|The Lord of the Rings Online            |videoGame|2007     |7.8          |564     |
|Cobra Kai: The Karate Kid Saga Continues|videoGame|2020     |7.6          |329     |
|Chivalry II                             |videoGame|2021     |6.8          |214     |
+----------------------------------------+---------+---------+-------------+--------+

Total number of records in the result: 3
Exporting results with automatically determined 1 partitions


## №4: Top 3 TV series by rating and number of votes for each region of localization

### №4: Топ-3 серіали (tvSeries) за рейтингом та кількістю голосів для кожного регіону

In [ ]:
def top_rated_series_by_region(df_basics: DataFrame, df_ratings: DataFrame, df_akas: DataFrame) -> DataFrame:

    series_with_ratings = df_basics \
        .join(df_ratings, "tconst", "inner") \
        .filter(
            (F.col("averageRating") >= 7) &
            (F.col("numVotes") >= 1000)
        )

    series_with_ratings = series_with_ratings.filter(F.col("titleType") == "tvSeries")

    series_with_region = series_with_ratings \
        .join(df_akas, df_basics["tconst"] == df_akas["titleId"], "inner") \
        .filter(df_akas["region"].isNotNull())

    window_spec = Window \
        .partitionBy("region") \
        .orderBy(F.desc("averageRating"), F.desc("numVotes"), F.asc("primaryTitle"))

    ranked_series = series_with_region.withColumn("row_num", F.row_number().over(window_spec))

    top_3_series = ranked_series.filter(F.col("row_num") <= 3)

    result = top_3_series.select("region", "primaryTitle", "averageRating", "numVotes")

    # result.show(20, truncate=False)
    return result

In [ ]:
result = top_rated_series_by_region(dataframes["basics"], dataframes["ratings"], dataframes["akas"])
export_result(result,"/data/results/top_rated_series_by_region.csv",title="Top-3 TV series for each region")

Results for: Top-3 TV series for each region
+------+------------------------------+-------------+--------+
|region|primaryTitle                  |averageRating|numVotes|
+------+------------------------------+-------------+--------+
|AE    |Breaking Bad                  |9.5          |2313754 |
|AE    |De'ah Da'iah                  |9.5          |1161    |
|AE    |Tree of Life                  |9.4          |17885   |
|AF    |The Other Me                  |8.9          |7612    |
|AF    |The Blacklist                 |7.9          |289109  |
|AL    |Game of Thrones               |9.2          |2421554 |
|AL    |The Island                    |9.1          |3207    |
|AL    |Friends                       |8.9          |1139938 |
|AM    |The Fairly OddParents         |7.2          |49255   |
|AR    |Breaking Bad                  |9.5          |2313754 |
|AR    |Avatar: The Last Airbender    |9.3          |403535  |
|AR    |When Life Gives You Tangerines|9.3          |11629   |
|AT    |Fr

## №5: The longest movie of each genre

### №5: Найдовші фільми в кожному жанрі

In [ ]:
def longest_movie_per_genre(df_basics: DataFrame) -> DataFrame:

    filtered_movies = df_basics \
        .filter(
            (F.col("titleType") == "movie") &
            (F.col("runtimeMinutes").isNotNull()) &
            (F.col("genres").isNotNull())
        )

    exploded = filtered_movies.withColumn("genre", F.explode("genres"))

    window_spec = Window \
        .partitionBy("genre") \
        .orderBy(F.desc("runtimeMinutes"))

    ranked = exploded.withColumn("rank", F.row_number().over(window_spec))

    top_movies = ranked.filter(F.col("rank") == 1)

    result = top_movies.select("genre", "primaryTitle", "runtimeMinutes")

    # result.show(20, truncate=False)
    return result

In [ ]:
result = longest_movie_per_genre(dataframes["basics"])
export_result(result,"/data/results/longest_movie_per_genre.csv",title="The longest movie of each genre")

Results for: The longest movie of each genre
+-----------+--------------------------------+--------------+
|genre      |primaryTitle                    |runtimeMinutes|
+-----------+--------------------------------+--------------+
|Action     |h36:                            |2160          |
|Adult      |16 Hours of MILFS 3             |960           |
|Adventure  |Fireside Reading of Dracula     |565           |
|Animation  |100                             |59460         |
|Biography  |O.J.: Made in America           |467           |
|Comedy     |London EC1                      |5460          |
|Crime      |Testing Movie1                  |1234          |
|Documentary|Logistics                       |51420         |
|Drama      |Qw                              |10062         |
|Family     |World Peace & Prayer Day        |2400          |
|Fantasy    |Funusion                        |1440          |
|Film-Noir  |Not as a Stranger               |135           |
|Game-Show  |The Netflix 

## №6: TV series with the highest number of episodes and their average rating

### №6: Які телевізійні серіали мають найбільшу кількість епізодів та який їхній середній рейтинг?

In [ ]:
def longest_series_with_ratings(df_basics: DataFrame, df_episode: DataFrame, df_ratings: DataFrame) -> DataFrame:

    df_basics_renamed = df_basics.withColumnRenamed("tconst", "series_tconst")

    df_series_episodes = df_basics_renamed.join(
        df_episode,
        df_episode["parentTconst"] == df_basics_renamed["series_tconst"],
        "left_outer"
    )

    series_episode_count = df_series_episodes.groupBy("primaryTitle", "series_tconst") \
        .agg(F.count("episodeNumber").alias("num_episodes"))

    df_with_ratings = series_episode_count.join(
        df_ratings,
        series_episode_count["series_tconst"] == df_ratings["tconst"],
        "inner"
    )

    result = df_with_ratings.select(
        "primaryTitle", "num_episodes", "averageRating"
    ).orderBy(F.desc("num_episodes"))

    # result.show(20, truncate=False)
    return result

In [ ]:
result = longest_series_with_ratings(dataframes["basics"], dataframes["episode"], dataframes["ratings"])
export_result(result,"/data/results/longest_series_with_ratings.csv",title="Longest series with ratings")

Results for: Longest series with ratings
+-----------------------------+------------+-------------+
|primaryTitle                 |num_episodes|averageRating|
+-----------------------------+------------+-------------+
|Days of Our Lives            |15209       |5.3          |
|The Young and the Restless   |13254       |5.3          |
|Coronation Street            |10976       |5.6          |
|As the World Turns           |10916       |6.3          |
|General Hospital             |10853       |6.6          |
|Emmerdale Farm               |10176       |4.9          |
|Barátok közt                 |10118       |2.3          |
|Eat Bulaga                   |9948        |7.7          |
|ASAP                         |9673        |6.5          |
|The Bold and the Beautiful   |9515        |3.5          |
|Neighbours                   |9258        |5.6          |
|Jeopardy!                    |9256        |8.3          |
|Guiding Light                |9164        |6.6          |
|Search for Tom